In [11]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import random

DATASET_PATH = "/kaggle/input/deep911/validation/combi"
LABEL_FILE_PATH = "/kaggle/input/deep911/validation/label_valid.txt"
NUM_CLASSES = 2
SAMPLE_RATE = 16000
DURATION = 5
N_MELS = 128
MAX_TIME_STEPS = 109


In [12]:
def augment_audio(audio):
    augmented_audio = audio.copy()

    shift_range = int(SAMPLE_RATE * 0.5)
    shift = random.randint(-shift_range, shift_range)
    augmented_audio = np.roll(augmented_audio, shift)

  
    if random.random() > 0.5:
        n_steps = random.randint(-2, 2)
        augmented_audio = librosa.effects.pitch_shift(augmented_audio, sr=SAMPLE_RATE, n_steps=n_steps)

   
    noise = np.random.randn(len(audio)) * 0.003
    augmented_audio += noise

   
    if random.random() > 0.5:
        stretch_factor = random.uniform(0.8, 1.2)
        augmented_audio = librosa.effects.time_stretch(augmented_audio, rate=stretch_factor)

    return augmented_audio


In [15]:

labels = {}
with open(LABEL_FILE_PATH, 'r') as label_file:
    lines = label_file.readlines()
for line in lines:
    parts = line.strip().split()
    if len(parts) < 3:
        continue
    file_name = parts[1]
    label = 1 if parts[-1] == "bonafide" else 0
    labels[file_name] = label


In [16]:
X = []
y = []
for file_name, label in labels.items():
    file_path = os.path.join(DATASET_PATH, file_name + ".flac")
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    
   
    for a in [audio, augment_audio(audio), augment_audio(audio)]:
        mel_spectrogram = librosa.feature.melspectrogram(y=a, sr=SAMPLE_RATE, n_mels=N_MELS)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        if mel_spectrogram.shape[1] < MAX_TIME_STEPS:
            mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, MAX_TIME_STEPS - mel_spectrogram.shape[1])), mode='constant')
        else:
            mel_spectrogram = mel_spectrogram[:, :MAX_TIME_STEPS]
        X.append(mel_spectrogram)
        y.append(label)

X = np.array(X)
y = np.array(y)

y_encoded = to_categorical(y, NUM_CLASSES)
split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y_encoded[:split_index], y_encoded[split_index:]


/tmp/ipykernel_30/2963867113.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [17]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (25924, 128, 109)
y_train shape: (25924, 2)


In [24]:
#CNN model
input_shape = (N_MELS, MAX_TIME_STEPS, 1)
model_input = Input(shape=input_shape)

x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(model_input)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.4)(x)

x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.4)(x)

x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.5)(x)

x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.5)(x)

x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
model_output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=model_input, outputs=model_output)

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

def scheduler(epoch, lr):
    return lr * tf.math.exp(-0.1).numpy() if epoch >= 10 else lr

callback = LearningRateScheduler(scheduler)

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(X_train[..., np.newaxis], y_train, 
                    batch_size=16, 
                    epochs=100, 
                    validation_data=(X_val[..., np.newaxis], y_val), 
                    callbacks=[callback, early_stopping])

model.save("/kaggle/working/kk-main-model/kk_main_final.h5")

Epoch 1/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8844 - loss: 0.5756

I0000 00:00:1730923411.565768     158 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_3', 36 bytes spill stores, 36 bytes spill loads



1621/1621 ━━━━━━━━━━━━━━━━━━━━ 51s 17ms/step - accuracy: 0.8844 - loss: 0.5755 - val_accuracy: 0.9977 - val_loss: 0.3325 - learning_rate: 5.0000e-05
Epoch 2/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9371 - loss: 0.4314 - val_accuracy: 0.9929 - val_loss: 0.3112 - learning_rate: 5.0000e-05
Epoch 3/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9509 - loss: 0.3727 - val_accuracy: 0.9954 - val_loss: 0.2665 - learning_rate: 5.0000e-05
Epoch 4/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9564 - loss: 0.3340 - val_accuracy: 0.9863 - val_loss: 0.2541 - learning_rate: 5.0000e-05
Epoch 5/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9628 - loss: 0.2989 - val_accuracy: 0.9841 - val_loss: 0.2424 - learning_rate: 5.0000e-05
Epoch 6/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9681 - loss: 0.2684 - val_accuracy: 0.9556 - val_loss: 0.2707 - learning_rate: 5.0000e-05
Epoch 7/100
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/

In [26]:

model.save("/kaggle/working/kk-main-model/kk_main_ksk.h5")


In [ ]:
print(f"Length of y_true: {len(y_true)}")
print(f"Length of y_pred_classes: {len(y_pred_classes)}")


In [21]:
# Testing
TEST_DATASET_PATH = "/kaggle/input/deep911/validation/combi"
test_files = os.listdir(TEST_DATASET_PATH)
y_true = np.array([labels[os.path.splitext(file)[0]] for file in test_files if os.path.splitext(file)[0] in labels])

X_test = []
for file_name in test_files:
    file_path = os.path.join(TEST_DATASET_PATH, file_name)
    try:
        audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max(0, MAX_TIME_STEPS - mel_spectrogram.shape[1]))), mode='constant')
        X_test.append(mel_spectrogram[:, :MAX_TIME_STEPS] / 80.0)
    except Exception as e:
        print(f"Error loading {file_name}: {e}")
        continue

X_test = np.array(X_test)[..., np.newaxis]

if len(X_test) > 0:
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)


/tmp/ipykernel_30/2570283783.py:10: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


KeyboardInterrupt: 

In [ ]:
model.summary()


In [ ]:

cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["spoof", "bonafide"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()
